# XGBoost = Extreme Gradient Boosting 모델


## XGBoost에서 개선된 점
- 자체적으로 결측치를 처리한다
- 속도를 향상한다. 
  - 근사 분할 탐색 알고리즘: 평범한 결정 트리는 그리디 알고리즘을 사용하지만, xgboost에서는 데이터를 나누는 퍼센트인 분위수 (퀀타일)을 사용해서 후보 분할을 제안한다. 
  - 희소성 고려 분할 탐색
  - 병렬 컴퓨팅: 부스팅은 이전 트리의 결과에 의존하기 때문에 병렬 컴퓨팅이 어렵다. 병렬 컴퓨팅이란 여러 개의 계산 유닛이 동시에 동일한 작업을 하는 것인데 xgboost는 데이터를 블록이라는 단위로 정렬하고 압축하여 여러 대의 머신이나 외부 메모리에 분산시킨다. 
  - 캐시 고려 접근: 캐시를 고려한 프리페칭을 사용한다. 
  - 블록 압축, 블록 샤딩
- 정확도를 향상한다.
  - 자체적으로 regularization 추가: 분산을 줄이고 과대적합을 방지한다. 

## 하이퍼파라미터 목록
| 이름              | 설명                                                                            | 기본값 |
|-------------------|---------------------------------------------------------------------------------|--------|
| n_estimators      | 앙상블의 트리 개수                                                              | 100    |
| learning_rate     | 부스팅의 각 단계에서 트리의 기여도를 조절                                       | 0.3    |
| max_depth         | 개별 트리의 최대 성장 가능한 깊이. 제한하면 과대적합 방지                       | 6      |
| gamma             | 라그랑주 승수. 노드 분할을 위한 최소 손실 감소를 지정한다. 늘리면 과대적합 방지 | 0      |
| min_child_weight  | 노드를 분할하기 위해 필요한 최소 가중치 합. 늘리면 과대적합 방지                | 1      |
| subsample         | 부스팅의 각 단계에서 사용되는 훈련 샘플의 비율 제한. 줄이면 과대적합 방지       | 1      |
| colsample_bytree  | 부스팅의 각 단계에서 사용되는 feature의 비율을 제한. 줄이면 과대적합 방지       | 1      |
| colsample_bylevel | 트리의 깊이마다 사용되는 feature의 비율 제한. 줄이면 과대적합 방지              | 1      |
| colsample_bynode  | 노드를 분할할 때마다 사용되는 feature의 비율 제한. 줄이면 과대적합 방지         | 1      |
| scale_pos_weight  | 언밸런스한 데이터에 대해 양성샘플과 음성샘플의 가중치 비율을 설정할 수 있다     | 1      |
| lambda            | 가중치 l2 규제를 통해 과대적합 방지                                             | 1      |
| alpha             | 가중치 l1 규제를 통해 과대적합 방지                                             | 0      |
| missing           | 누락된 값을 이 수치로 대체함.                                                   | None   |

In [2]:
# 경고 끄기
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
xgb.set_config(verbosity=0)

import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
# cross_val_score를 임포트합니다.
from sklearn.model_selection import cross_val_score

# numpy를 임포트합니다.
import numpy as np

df = pd.read_csv('../data/heart_disease.csv')

# 데이터를 X와 y로 나눕니다.
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

# 교차 검증 점수를 구합니다.
scores = cross_val_score(model, X, y, cv=5)

# 정확도를 출력합니다.
print('정확도:', np.round(scores, 2))

# 정확도 평균을 출력합니다.
print('정확도 평균: %0.2f' % (scores.mean()))

정확도: [0.84 0.85 0.82 0.8  0.77]
정확도 평균: 0.81


In [4]:
# GridSearchCV를 임포트합니다.
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

def grid_search(params, random=False): 

    xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', 
                        random_state=2, verbosity=0, use_label_encoder=False)
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, 
                                  n_jobs=-1, random_state=2)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    
    # X와 y에서 하이퍼파라미터 튜닝을 수행합니다.
    grid.fit(X, y)

    # 최상의 매개변수를 추출합니다.
    best_params = grid.best_params_

    # 최상의 매개변수를 출력합니다.
    print("최상의 매개변수:", best_params)
    
    # 최상의 점수를 추출합니다.
    best_score = grid.best_score_

    # 최상의 점수를 출력합니다.
    print("최상의 점수: {:.5f}".format(best_score))

grid_search(params={'n_estimators':[100, 200, 400, 800]})

In [6]:
# train_test_split 함수를 임포트합니다.
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

model = XGBClassifier(booster='gbtree', objective='binary:logistic')
eval_set = [(X_test, y_test)]
eval_metric="error"
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, 
          early_stopping_rounds=10, verbose=True)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
정확도: 89.47%
